In [1]:
from __future__ import print_function 

#%load_ext autoreload
#%autoreload 2

from matplotlib import pyplot as plt
%matplotlib inline

import os, sys
import numpy as np
import time

import tensorflow as tf
from tensorflow.keras import backend as K
K.clear_session()
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers import deserialize, serialize
from tensorflow.python.keras.saving import saving_utils

# Project imports 
from data import mnist,mnist_m
from data.tasks import *
from data.xray import * # temporary
from data.label_shift import *
from experiments.models import *
from experiments.training import *
from util.kl import *
from util.misc import *


#reset Keras Session
def reset_keras():
    sess = tf.compat.v1.keras.backend.get_session()
    tf.compat.v1.keras.backend.clear_session()
    sess.close()
    sess = tf.compat.v1.keras.backend.get_session()


    # use the same config as you used to create the session
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))
    
reset_keras()

# Load data & Train

In [2]:

alphas=[0,0.3]
architectures=["lenet"]#"fc", "resnet"
tasks=[2]
seeds=[1]
image_size=32
batch_size=32
from data.xray import *
import time
for t in tasks:
    for arch in architectures:
        for alpha in alphas:
            for seed in seeds:
                if t==7:
                        source_generator, target_generator=load_task(t,architecture=arch,binary=True,image_size=image_size,batch_size=batch_size)
                else:
                        x_source,y_source,x_target,y_target,_, _=load_task(task=t,architecture=arch,binary=True)
                if alpha==0:
                    pass
                else:
                    if t==7:
                        prior_generator, bound_generator, target_generator=load_task(task=t,alpha=alpha,architecture=arch,binary=True,image_size=image_size,seed=seed) ## create bound/prior split here as own generators
                        w_a=train_prior(alpha,num_priors,generator=prior_generator,val_generator=target_generator,save=True,task=t,binary=True,batch_size=batch_size,architecture=arch,seed=seed,image_size=image_size)
                    else:
                        print("Alpha is:"+str(alpha))
                        w_a=train_prior(alpha,1,x_train=x_source,y_train=y_source,x_target=x_target,y_target=y_target,save=True,task=t,binary=True,batch_size=batch_size,architecture=arch,seed=seed,image_size=image_size)
                if t==7:
                    if alpha!=0:
                        source_generator, target_generator=load_task(t,architecture=arch,binary=True,image_size=image_size)
                    print(len(source_generator))
                    w_s=train_posterior(alpha,generator=source_generator,val_generator=target_generator,prior_weights=None,save=True,task=t,binary=True,batch_size=batch_size,architecture=arch,seed=seed,image_size=image_size)
                else:
                    w_s=train_posterior(alpha,x_train=x_source,y_train=y_source,prior_weights=None,x_test=x_source,y_test=y_source,save=True,task=t,binary=True,batch_size=batch_size,architecture=arch,seed=seed,image_size=image_size)


mean, variance 0.36348352 70.18035
---------------Load MNIST----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)


mean, variance 1.1809415 74.36859
---------------Load MNIST-M----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)
[array([6328., 6565., 5243., 4761., 3981., 3157., 2866., 2431., 1707.,
       1160.], dtype=float32), array([ 576., 1313., 1748., 2381., 2844., 3157., 4012., 4862., 5119.,
       5799.], dtype=float32), array([ 575., 1312., 1747., 2380., 2843., 3156., 4010., 4862., 5118.,
       5798.], dtype=float32), array([6327., 6564., 5242., 4760., 3980., 3156., 2864., 2431., 1706.,
       1159.], dtype=float32)]
[[0.09102402, 0.2, 0.3333969, 0.500105, 0.7143934, 1.0, 1.3998604, 2.0, 2.9988284, 4.999138], [11.003478, 5.003049, 3.0005724, 2.0, 1.3999296, 1.0, 0.71421444, 0.5, 0.33333334, 0.19989651]]
[0.2        4.999138   0.3333969  ... 0.19989651 3.0005724  0.71421444]
Mean of 

In [3]:
#model.evaluate(target_generator,steps=target_generator.steps, workers=16)